In [1]:
import numpy as np
import pandas as pd

In [2]:
# Generate example data
date_range = pd.date_range(start='2024-01-01', periods=2_000, freq='d')
data = np.random.normal(50, 8, 2000)
df = pd.DataFrame(data, index=date_range, columns=['value'])

In [3]:
df.head()

,value
2024-01-01,55.497380
2024-01-02,25.308695
2024-01-03,58.908446
2024-01-04,43.740365
2024-01-05,59.042182


In [4]:
from spyc import SPC, x_chart, mr_chart, plotly_chart

In [5]:
spc_example = SPC(data_in=df, 
                  target_col = 'value',
                  # Default values to None (will use all available data for control line calc.)
                  fix_control_start_dt=None,
                  fix_control_end_dt=None,
                  # Data shows repeated patterns every 7 days
                  seasonal=7
                 )

In [6]:
# OPTIONAL. i.e., new beds added and control lines need to be re-calculated
# (can be done once, or multiple times)
spc_example.add_process_change_date(change_date='2025-01-21',
                                    fix_control_start_dt='2025-01-27',
                                    fix_control_end_dt='2028-09-27',
                                    process_name='New ward')

In [7]:
x_chart_data = spc_example.calculate_spc(spc_calc_func=x_chart,
                          rule_1=True) 
x_chart_data.head()

,CL,LCL,UCL,process,period_name,season,Rule 1 violation
2024-01-01,49.298882,25.196673,73.401091,55.497380,NaN,0,0
2024-01-02,48.187679,20.001857,76.373502,25.308695,NaN,1,0
2024-01-03,50.475782,28.544718,72.406846,58.908446,NaN,2,0
2024-01-04,49.291444,24.491161,74.091727,43.740365,NaN,3,0
2024-01-05,49.265350,24.002118,74.528582,59.042182,NaN,4,0


In [11]:
plotly_chart(x_chart_data)

In [9]:
# SPC data/chart stored for specific SPC function. The function can be custom/use built in 
# but requires a dictionary with 'prococes', 'CL', 'LCL', 'UCL' to be returned

def quantile_chart_calc(data, target_col, cl_start_dt=None, cl_end_dt=None):
    
    CL = data[target_col].quantile(0.5)
    LCL = data[target_col].quantile(0.01)
    UCL = data[target_col].quantile(0.99)
    
    return {'process': data[target_col], 'CL': CL, 'LCL': LCL, 'UCL': UCL}
    
# Specific rules can then be defined for each chart type
custom_chart_data = spc_example.calculate_spc(spc_calc_func=quantile_chart_calc,
                                              rule_1=True) 

In [12]:
plotly_chart(custom_chart_data)